## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import spearmanr
from IPython.display import clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt

## 2. Load OTU Data

In [ ]:
# Load OTU abundance table
otu = pd.read_csv(
    "india_species_abundance_clean.tsv",
    sep="\t",
    index_col=0
)

otu.head()

In [ ]:
# Mean abundance per species
baseline_abundance = otu.mean(axis=1)
baseline_abundance.sort_values(ascending=False).head(10)

## 3. Spearman Correlation and Bootstrap Consensus

In [ ]:
# Compute Spearman correlation matrix
corr, pval = spearmanr(otu.T)

corr_df = pd.DataFrame(
    corr,
    index=otu.index,
    columns=otu.index
)

pval_df = pd.DataFrame(
    pval,
    index=otu.index,
    columns=otu.index
)

print("Correlation matrix shape:", corr_df.shape)

In [ ]:
def bootstrap_spearman(data, n_boot=100, alpha=0.05):
    """Generate consensus edge network using bootstrap Spearman correlations"""
    species = data.index
    edge_counts = {}

    for _ in range(n_boot):
        boot = data.sample(frac=1, replace=True, axis=1)
        corr, pval = spearmanr(boot.T)

        corr = pd.DataFrame(corr, index=species, columns=species)
        pval = pd.DataFrame(pval, index=species, columns=species)

        for i in species:
            for j in species:
                if i != j and pval.loc[i, j] < alpha:
                    sign = np.sign(corr.loc[i, j])
                    edge_counts[(i, j)] = edge_counts.get((i, j), 0) + sign

    return edge_counts

edge_counts = bootstrap_spearman(otu)
print(f"Consensus edges generated: {len(edge_counts)}")

## 4. Build Interaction Network (SPIEC-EASI)

In [ ]:
G = nx.DiGraph()

for (i, j), score in edge_counts.items():
    weight = score / 100  # normalize
    if abs(weight) > 0.1:  # consensus threshold
        G.add_edge(i, j, weight=weight)

print(f"Interaction network: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## 5. Define Functional Traits (AGORA-inspired)

In [ ]:
# Reference traits for known species (will be expanded via genus mapping)
traits_ref = pd.DataFrame({
    "SCFA": [1, 0, 1, 0],
    "Siderophore": [0, 1, 0, 1],
    "Vitamin_Biosynthesis": [1, 0, 1, 0],
    "pH_reduction": [1, 0, 0, 0],
    "Barrier_support": [1, 0, 1, 0]
}, index=[
    "Lactobacillus_plantarum",
    "Escherichia_coli",
    "Bifidobacterium_longum",
    "Enterobacter_cloacae"
])

traits_ref

## 6. Nutrient Model

In [ ]:
nutrients = {
    "Iron": {"SCFA":0.4,"pH_reduction":0.3,"Barrier_support":0.2,"Siderophore":-0.6},
    "Vitamin_B12": {"Vitamin_Biosynthesis":0.7,"Barrier_support":0.2},
    "Folate": {"Vitamin_Biosynthesis":0.6},
    "Calcium": {"SCFA":0.5,"pH_reduction":0.4},
    "Magnesium": {"SCFA":0.4},
    "Zinc": {"Barrier_support":0.4,"Siderophore":-0.5}
}

print("Nutrient model defined with", len(nutrients), "nutrients")

## 7. Absorption Score Function

In [ ]:
def absorption_score(abundance, nutrient, traits_table):
    """Calculate nutrient absorption score based on microbial composition"""
    score = 0
    for sp, ab in abundance.items():
        if sp in traits_table.index:
            for trait, w in nutrients[nutrient].items():
                if trait in traits_table.columns:
                    score += ab * traits_table.loc[sp, trait] * w
    return score

## 8. Genus-Level Functional Priors

In [ ]:
# Extract genus from species names
species_list = baseline_abundance.index

def get_genus(species):
    return species.split("_")[0]

genus_map = {sp: get_genus(sp) for sp in species_list}

# Genus-level functional priors (confidence 0–1)
genus_traits = {
    "Lactobacillus": {
        "SCFA": 0.9,
        "pH_reduction": 0.9,
        "Barrier_support": 0.8,
        "Vitamin_Biosynthesis": 0.6,
        "Siderophore": 0.0
    },
    "Bifidobacterium": {
        "SCFA": 0.8,
        "pH_reduction": 0.7,
        "Barrier_support": 0.8,
        "Vitamin_Biosynthesis": 0.7,
        "Siderophore": 0.0
    },
    "Bacteroides": {
        "SCFA": 0.6,
        "pH_reduction": 0.3,
        "Barrier_support": 0.4,
        "Vitamin_Biosynthesis": 0.3,
        "Siderophore": 0.4
    },
    "Prevotella": {
        "SCFA": 0.7,
        "pH_reduction": 0.4,
        "Barrier_support": 0.4,
        "Vitamin_Biosynthesis": 0.3,
        "Siderophore": 0.3
    },
    "Escherichia": {
        "SCFA": 0.1,
        "pH_reduction": 0.0,
        "Barrier_support": 0.1,
        "Vitamin_Biosynthesis": 0.1,
        "Siderophore": 0.9
    },
    "Enterobacter": {
        "SCFA": 0.1,
        "pH_reduction": 0.0,
        "Barrier_support": 0.1,
        "Vitamin_Biosynthesis": 0.1,
        "Siderophore": 0.8
    }
}

print(f"Genus priors defined for {len(genus_traits)} genera")

In [ ]:
# Auto-generate species-level trait table from genus priors
all_traits = []

for sp, genus in genus_map.items():
    if genus in genus_traits:
        row = genus_traits[genus].copy()
    else:
        # Default weak functionality (unknown genus)
        row = {
            "SCFA": 0.2,
            "pH_reduction": 0.1,
            "Barrier_support": 0.1,
            "Vitamin_Biosynthesis": 0.1,
            "Siderophore": 0.2
        }
    row["Species"] = sp
    all_traits.append(row)

traits = pd.DataFrame(all_traits).set_index("Species")
traits.head()

## 9. Baseline Nutrient Absorption

In [ ]:
for n in nutrients:
    score = absorption_score(baseline_abundance, n, traits)
    print(f"{n}: {round(score, 3)}")

## 10. Microbe Addition Simulation

In [ ]:
def simulate_addition(abundance, microbe, delta):
    """Simulate addition of a microbe and propagate effects through network"""
    new = abundance.copy()
    new[microbe] = new.get(microbe, 0) + delta

    if microbe in G:
        for tgt in G.successors(microbe):
            new[tgt] += delta * G[microbe][tgt]["weight"]

    new[new < 0] = 0
    return new

# Example: add Lactobacillus
new_abundance = simulate_addition(
    baseline_abundance,
    list(baseline_abundance.index)[0] if len(baseline_abundance) > 0 else "Lactobacillus_plantarum",
    delta=0.02
)

print("\nNutrient changes after microbe addition:")
for n in nutrients:
    before = absorption_score(baseline_abundance, n, traits)
    after = absorption_score(new_abundance, n, traits)
    print(f"{n}: {round(before, 3)} → {round(after, 3)} (Δ {round(after-before, 3)})")

## 11. Nutrient Targeting - Microbe Recommendations

In [ ]:
def recommend_microbes(nutrient, top=3):
    """Recommend microbes to increase or decrease for a nutrient"""
    pos = {}
    neg = {}

    for sp in traits.index:
        score = sum(
            traits.loc[sp, t] * w
            for t, w in nutrients[nutrient].items()
            if t in traits.columns
        )

        if score > 0:
            pos[sp] = score
        elif score < 0:
            neg[sp] = score

    inc = sorted(pos.items(), key=lambda x: x[1], reverse=True)[:top]
    dec = sorted(neg.items(), key=lambda x: x[1])[:top]

    return inc, dec

inc, dec = recommend_microbes("Iron")

print("Iron absorption - recommended changes:")
print("\nIncrease:")
for sp, sc in inc:
    print(f"  {sp}: {sc}")

print("\nDecrease:")
for sp, sc in dec:
    print(f"  {sp}: {sc}")

## 12. Interactive Simulation UI

In [ ]:
available_bacteria = sorted(
    list(set(baseline_abundance.index).intersection(traits.index))
)
available_nutrients = list(nutrients.keys())

bacteria_dropdown = widgets.Dropdown(
    options=available_bacteria,
    description="Bacteria:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="400px")
)

nutrient_dropdown = widgets.Dropdown(
    options=available_nutrients,
    description="Nutrient:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="300px")
)

delta_input = widgets.FloatText(
    value=0.02,
    description="Δ Abundance:",
    step=0.01,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="300px")
)

run_button = widgets.Button(
    description="Run Simulation",
    button_style="success"
)

output_box = widgets.Output()

def run_simulation(button):
    with output_box:
        clear_output()
        microbe = bacteria_dropdown.value
        nutrient = nutrient_dropdown.value
        delta = delta_input.value

        before = absorption_score(baseline_abundance, nutrient, traits)
        new_abundance = simulate_addition(baseline_abundance, microbe, delta)
        after = absorption_score(new_abundance, nutrient, traits)

        print("🧬 Simulation Result")
        print("-" * 27)
        print(f"Bacteria added : {microbe}")
        print(f"Nutrient       : {nutrient}")
        print(f"Δ Abundance    : {delta}")
        print()
        print(f"Absorption BEFORE : {round(before, 4)}")
        print(f"Absorption AFTER  : {round(after, 4)}")
        print(f"Change (Δ)        : {round(after - before, 4)}")

run_button.on_click(run_simulation)

display(
    widgets.VBox([
        bacteria_dropdown,
        nutrient_dropdown,
        delta_input,
        run_button,
        output_box
    ])
)

## 13. Diet Mapping

In [ ]:
diet_map = {
    "Dietary_Fiber": {
        "Bifidobacterium": 0.9,
        "Lactobacillus": 0.7,
        "Prevotella": 0.8,
        "Bacteroides": 0.6
    },
    "Resistant_Starch": {
        "Ruminococcus": 0.8,
        "Eubacterium": 0.7,
        "Bifidobacterium": 0.6
    },
    "Fermented_Foods": {
        "Lactobacillus": 0.9,
        "Bifidobacterium": 0.7
    },
    "Red_Meat": {
        "Escherichia": 0.6,
        "Enterobacter": 0.7
    },
    "Polyphenols": {
        "Akkermansia": 0.8,
        "Bifidobacterium": 0.6
    }
}

def simulate_diet(abundance, diet, grams_per_day, days):
    """Simulate dietary intervention effects on microbiome"""
    new = abundance.copy()
    intensity = (grams_per_day / 50) * (days / 14)
    intensity = min(intensity, 2.0)

    for sp in new.index:
        genus = sp.split("_")[0]
        if genus in diet_map.get(diet, {}):
            growth = diet_map[diet][genus]
            new[sp] += new[sp] * growth * intensity * 0.1

    return new

## 14. Time-Dynamic Microbiome Model

In [ ]:
# Prepare base data
df = pd.read_csv("india_species_abundance_clean.tsv", sep="\t", index_col=0)

# Auto-fix orientation
if not any("_" in c for c in df.columns[:10]):
    df = df.T

# Normalize
df = df.div(df.sum(axis=1), axis=0)
bacteria = df.columns.tolist()

print(f"Data loaded: {df.shape[0]} samples, {df.shape[1]} bacteria")

In [ ]:
# Build interaction network from data
interaction_matrix = df.corr(method="spearman").fillna(0)
interaction_matrix[interaction_matrix.abs() < 0.3] = 0

# Create trait table
traits_dyn = pd.DataFrame(index=bacteria)
traits_dyn["scfa"] = interaction_matrix.mean(axis=1)
traits_dyn["fiber_fermentation"] = df.mean()
traits_dyn["pathogenicity"] = -interaction_matrix.min(axis=1)
traits_dyn = traits_dyn.fillna(0)

print("Traits computed for dynamic model")

In [ ]:
# Nutrient model for dynamic simulation
nutrients_dyn = {
    "Iron": {"scfa": 0.6, "fiber_fermentation": 0.7, "pathogenicity": -0.8},
    "Vitamin_B12": {"scfa": 0.7, "pathogenicity": -0.6},
    "Folate": {"fiber_fermentation": 0.8, "scfa": 0.4},
    "Calcium": {"scfa": 0.5},
    "Zinc": {"fiber_fermentation": 0.7, "pathogenicity": -0.6}
}

# Simulation parameters
TIME_STEPS = 30
BOOTSTRAPS = 50
growth_rate = 0.05
interaction_strength = 0.1

B0 = df.mean()
B0 = B0 / B0.sum()

print(f"Simulation parameters set: {TIME_STEPS} days, {BOOTSTRAPS} bootstraps")

In [ ]:
def run_dynamic_simulation(target, delta):
    """Run time-dynamic microbiome simulation with bootstrap uncertainty"""
    nutrient_runs = []

    for _ in range(BOOTSTRAPS):
        # Add interaction noise
        noise = np.random.normal(0, 0.05, interaction_matrix.shape)
        A = interaction_matrix + noise

        B = B0.copy()
        B[target] = max(B[target] + delta, 0)
        B = B / B.sum()

        traj = []

        for _ in range(TIME_STEPS):
            Traits = traits_dyn.T @ B

            nutrient_vals = {
                n: np.tanh(sum(Traits[t] * w for t, w in weights.items()))
                for n, weights in nutrients_dyn.items()
            }
            traj.append(nutrient_vals)

            B = B + growth_rate * B + interaction_strength * (A @ B)
            B[B < 0] = 0
            B = B / B.sum()

        nutrient_runs.append(pd.DataFrame(traj))

    return nutrient_runs

print("Dynamic simulation function defined")

## 15. Interactive Dynamic Simulator

In [ ]:
bacteria_dd = widgets.Dropdown(
    options=bacteria,
    description="Perturb bacterium:",
    layout=widgets.Layout(width="500px")
)

delta_slider = widgets.FloatSlider(
    value=0.05, min=-0.2, max=0.2, step=0.01,
    description="Δ abundance:"
)

run_btn = widgets.Button(description="Run Simulation", button_style="success")
output = widgets.Output()

def run_model(_):
    with output:
        clear_output()

        target = bacteria_dd.value
        delta = delta_slider.value

        print("Running simulation...")
        runs = run_dynamic_simulation(target, delta)

        stacked = np.stack([r.values for r in runs])
        mean = stacked.mean(axis=0)
        std = stacked.std(axis=0)

        print(f"🧬 Target bacterium: {target}")
        print(f"Δ abundance: {delta}")
        print(f"Time steps: {TIME_STEPS}, Bootstraps: {BOOTSTRAPS}\n")

        plt.figure(figsize=(10,5))
        for i, nutrient in enumerate(runs[0].columns):
            plt.plot(mean[:,i], label=nutrient)
            plt.fill_between(
                range(TIME_STEPS),
                mean[:,i] - std[:,i],
                mean[:,i] + std[:,i],
                alpha=0.2
            )

        plt.axhline(0, linestyle="--", color="gray")
        plt.xlabel("Time (days)")
        plt.ylabel("Predicted absorption impact")
        plt.title("Dynamic nutrient response with uncertainty")
        plt.legend()
        plt.tight_layout()
        plt.show()

run_btn.on_click(run_model)

display(widgets.VBox([
    bacteria_dd,
    delta_slider,
    run_btn,
    output
]))

## 16. Dynamic Simulator with Diet Intervention

In [ ]:
diet_trait_effects = {
    None: {},
    "Dietary_fiber": {"fiber_fermentation": 0.5, "pathogenicity": -0.3},
    "Resistant_starch": {"scfa": 0.4, "fiber_fermentation": 0.2},
    "Fermented_foods": {"scfa": 0.3, "pathogenicity": -0.2},
    "Low_fat_high_fiber": {"fiber_fermentation": 0.4, "pathogenicity": -0.4},
    "High_protein": {"pathogenicity": 0.3},
}

diet_strength = 0.05

def apply_diet_bias(B, diet):
    """Apply dietary intervention bias to microbiome abundances"""
    if diet not in diet_trait_effects or diet is None:
        return B

    bias = pd.Series(0.0, index=B.index)
    for trait, w in diet_trait_effects[diet].items():
        if trait in traits_dyn.columns:
            bias += traits_dyn[trait] * w

    bias[bias < 0] = 0
    if bias.sum() > 0:
        bias = bias / bias.sum()

    return B + diet_strength * bias

print("Diet intervention function defined")

In [ ]:
def run_simulation_with_diet(target_bacteria, delta, diet):
    """Run simulation with diet intervention"""
    all_runs = []

    for _ in range(BOOTSTRAPS):
        A = interaction_matrix + np.random.normal(0, 0.05, interaction_matrix.shape)

        B = B0.copy()
        B[target_bacteria] = max(B[target_bacteria] + delta, 0)
        B = B / B.sum()

        nutrient_traj = []

        for _ in range(TIME_STEPS):
            Traits = traits_dyn.T @ B

            nutrient_vals = {
                n: np.tanh(sum(Traits[t] * w for t, w in weights.items()))
                for n, weights in nutrients_dyn.items()
            }
            nutrient_traj.append(nutrient_vals)

            B = B + growth_rate * B + interaction_strength * (A @ B)
            B = apply_diet_bias(B, diet)
            B[B < 0] = 0
            B = B / B.sum()

        all_runs.append(pd.DataFrame(nutrient_traj))

    return all_runs

print("Diet-integrated simulation function defined")

In [ ]:
bacteria_dd2 = widgets.Dropdown(
    options=bacteria,
    description="Perturb bacterium:",
    layout=widgets.Layout(width="500px")
)

delta_slider2 = widgets.FloatSlider(
    value=0.05, min=-0.2, max=0.2, step=0.01,
    description="Δ abundance:"
)

diet_dd = widgets.Dropdown(
    options=list(diet_trait_effects.keys()),
    description="Diet intervention:",
    layout=widgets.Layout(width="400px")
)

run_btn2 = widgets.Button(description="Run Simulation", button_style="success")
output2 = widgets.Output()

def run_model_with_diet(_):
    with output2:
        clear_output()

        target = bacteria_dd2.value
        delta = delta_slider2.value
        diet = diet_dd.value

        print("Running simulation...")
        runs = run_simulation_with_diet(target, delta, diet)

        stacked = np.stack([r.values for r in runs])
        mean = stacked.mean(axis=0)
        std = stacked.std(axis=0)

        print(f"🧬 Target bacterium: {target}")
        print(f"Δ abundance: {delta}")
        print(f"Diet: {diet}")
        print(f"Time: {TIME_STEPS} days | Bootstraps: {BOOTSTRAPS}\n")

        plt.figure(figsize=(10,5))
        for i, nutrient in enumerate(runs[0].columns):
            plt.plot(mean[:,i], label=nutrient)
            plt.fill_between(
                range(TIME_STEPS),
                mean[:,i] - std[:,i],
                mean[:,i] + std[:,i],
                alpha=0.2
            )

        plt.axhline(0, linestyle="--", color="gray")
        plt.xlabel("Time (days)")
        plt.ylabel("Predicted absorption impact")
        plt.title("Dynamic nutrient response with diet & microbiome perturbation")
        plt.legend()
        plt.tight_layout()
        plt.show()

run_btn2.on_click(run_model_with_diet)

display(
    widgets.VBox([
        bacteria_dd2,
        delta_slider2,
        diet_dd,
        run_btn2,
        output2
    ])
)